# Querying databases and converting to `pandas`

Main commands to:  
- Access and query an online database:  
  - When data are in `.json`, we use the python package `request`  
  - When data are in RDF triples, we use the python package `SPARQLWrapper`  
- Convert information from `.json` to `pandas` dataframe   

Examples are from this notebook: https://github.com/OpenRiskNet/notebooks/blob/master/BridgeDb/DataCure%2BAOPlink.ipynb  

# 1. `.json` using `requests`

`requests` is a package to access .json databases (https://realpython.com/python-requests/)

In [1]:
import requests

#### Using `get` to access database, fields, and values

In [2]:
# database url
chemidconvert = 'https://chemidconvert.cloud.douglasconnect.com/v1/'

In [3]:
# query the database (function: get())
response1 = requests.get(chemidconvert + 'name/to/smiles', 
                        params={'name': 'paracetamol'})

In [4]:
# if the query is successful, we get 200, otherwise 404 (function: status_code)
if response1.status_code == 200:
    print('Successful')
elif response1.status_code == 404:
    print('Not successful')

Successful


In [5]:
# extract field in .json format (functions: get(), .json())
response1 = requests.get(chemidconvert + 'name/to/smiles', params={'name': 'paracetamol'}).json()
print (response1) 

{'smiles': 'CC(=O)Nc1ccc(O)cc1'}


In [6]:
# extract field value (functions: get(), .json(), ['field_name'])
response1 = requests.get(chemidconvert + 'name/to/smiles', params={'name': 'paracetamol'}).json()['smiles']
print (response1) 

CC(=O)Nc1ccc(O)cc1


#### Using `params` to query the database  
Parameters can be passed as dictionary, list of tuples, or bytes. Here example with dictionary

In [7]:
# database url
tggatesconvert = 'http://open-tggates-api.cloud.douglasconnect.com/v2/'

In [8]:
# filter the data you want (functions: get() with keyword "params", followed by dictionary with parameter fields and corresponding values ) 
response2 = requests.get(tggatesconvert + 'samples',
                         params={'limit': 10000, 
                                 'compoundNameFilter': 'paracetamol',
                                 'organismFilter': 'Human', 
                                 'tissueFilter': 'Liver',
                                 'cellTypeFilter': 'in vitro', 
                                 'repeatTypeFilter': 'Single',
                                 'timepointHrFilter': '24.0', 
                                 'doseLevelFilter': 'High'
                          })
print(response2.json())

{'aggregations': {'compoundName': {'buckets': [{'doc_count': 2, 'key': '2,4-dinitrophenol'}, {'doc_count': 2, 'key': '2-nitrofluorene'}, {'doc_count': 1, 'key': 'LPS'}, {'doc_count': 2, 'key': 'N-methyl-N-nitrosourea'}, {'doc_count': 2, 'key': 'N-nitrosomorpholine'}, {'doc_count': 2, 'key': 'TNFalpha'}, {'doc_count': 2, 'key': 'WY-14643'}, {'doc_count': 2, 'key': 'acarbose'}, {'doc_count': 2, 'key': 'acetamide'}, {'doc_count': 2, 'key': 'acetamidofluorene'}, {'doc_count': 2, 'key': 'acetaminophen'}, {'doc_count': 2, 'key': 'acetazolamide'}, {'doc_count': 2, 'key': 'adapin'}, {'doc_count': 2, 'key': 'aflatoxin B1'}, {'doc_count': 2, 'key': 'ajmaline'}, {'doc_count': 2, 'key': 'allopurinol'}, {'doc_count': 2, 'key': 'allyl alcohol'}, {'doc_count': 2, 'key': 'alpidem'}, {'doc_count': 2, 'key': 'amiodarone'}, {'doc_count': 2, 'key': 'amitriptyline'}, {'doc_count': 2, 'key': 'amphotericin B'}, {'doc_count': 2, 'key': 'aspirin'}, {'doc_count': 2, 'key': 'azathioprine'}, {'doc_count': 2, 'key

# 2. `RDF` format using `SPARQLWrapper`

`SPARQLWrapper` is a package to access RDF databases (https://rdflib.github.io/sparqlwrapper/)

In [9]:
from SPARQLWrapper import SPARQLWrapper, JSON

#### Set database

In [10]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")

#### Query database using `SPARQL`

In [11]:
# create query
gene = "ENSG00000106258"
pathwayQuery = '''
      SELECT DISTINCT ?ensembl ?pathwayRes (str(?wpid) as ?pathway) (str(?title) as ?pathwayTitle)
      WHERE {{
        ?gene a wp:GeneProduct ;
          dcterms:identifier ?id ;
          dcterms:isPartOf ?pathwayRes ;
          wp:bdbEnsembl <http://identifiers.org/ensembl/{0}> .
        ?pathwayRes a wp:Pathway ;
          dcterms:identifier ?wpid ;
          dc:title ?title .
        BIND ( "gene" AS ?ensembl )
      }}
    '''.format(gene)

In [12]:
# set query
sparql.setQuery(pathwayQuery)

In [13]:
# set return format (JSON, XML, N3 - Note: the format must be the one imported: e.g. from SPARQLWrapper import SPARQLWrapper, JSON) 
sparql.setReturnFormat(JSON)

In [14]:
# execute query and format conversion
response3 = sparql.query().convert()

# 3. From `.json` to `pandas`

#### Example 1: Extracting the whole table from `.json` and convert it to `pandas`

In [15]:
import requests
import pandas

Using the `request` package, query the database  (`get()`)  and convert the answer to .json (`.json()`):

In [16]:
# database url
tggatesconvert = 'http://open-tggates-api.cloud.douglasconnect.com/v2/'

# compounds of interest
compoundset = {'paracetamol', 'acetominophen', 'methapyrilene', 'phenylbutazone', 'simvastatin', 'valproic acid'}
compounds_name = "|".join(compoundset) # separate with “|” for database specs 

# query database
response = requests.get(tggatesconvert + 'samples',
                        params={'limit': 10000, 
                                'compoundNameFilter': compounds_name,
                                'organismFilter': 'Human', 
                                'tissueFilter': 'Liver',
                                'cellTypeFilter': 'in vitro', 
                                'repeatTypeFilter': 'Single',
                                'timepointHrFilter': '24.0', 
                                'doseLevelFilter': 'High'
                          })
# convert to .json
samples = response.json() 

Convert from `.json` to `pandas` framework:

In [17]:
# convert to pandas
table = pandas.DataFrame(samples['samples'])
display(table)

,_id_,cellType,compoundName,controlSamples,doseLevel,organism,repeatType,sampleId,timepointHr,tissue
0,003016020014,in vitro,methapyrilene,[],High,Human,Single,003016020014,24.0,Liver
1,003016020015,in vitro,methapyrilene,[],High,Human,Single,003016020015,24.0,Liver
2,003016014002,in vitro,phenylbutazone,[],High,Human,Single,003016014002,24.0,Liver
3,003016014003,in vitro,phenylbutazone,[],High,Human,Single,003016014003,24.0,Liver
4,003016079002,in vitro,simvastatin,[],High,Human,Single,003016079002,24.0,Liver
5,003016079003,in vitro,simvastatin,[],High,Human,Single,003016079003,24.0,Liver
6,003016038010,in vitro,valproic acid,[],High,Human,Single,003016038010,24.0,Liver
7,003016038011,in vitro,valproic acid,[],High,Human,Single,003016038011,24.0,Liver


#### Example 2: Extracting one piece of information (i.e. one column) from `.json` and put it in a `pandas` framework

In [18]:
import requests
import pandas

Create the variables:

In [19]:
# database url
chemidconvert = 'https://chemidconvert.cloud.douglasconnect.com/v1/'

# compounds of interest
compoundset = {'paracetamol', 'acetominophen', 'methapyrilene', 'phenylbutazone', 'simvastatin', 'valproic acid'}

# create pandas framework
pandas.set_option('display.max_colwidth', -1)  # make table as wide as page
compounds = pandas.DataFrame(columns=['Compound name', 'Smiles', 'Image'])

In a `for` loop, query the database(`.get()`), convert to answer .json (`json()`), and then to `pandas` :

In [20]:
# query the database and fill out pandas framework 
for compound in compoundset:
    # query
    smiles = requests.get(chemidconvert + 'name/to/smiles', 
                          params={'name': compound}).json()['smiles']
    # fill out pandas framework
    compounds = compounds.append({'Compound name': compound, 'Smiles': smiles, 'Image': smiles}, ignore_index=True)
display (compounds)

,Compound name,Smiles,Image
0,valproic acid,CCCC(CCC)C(O)=O,CCCC(CCC)C(O)=O
1,phenylbutazone,CCCCC1C(=O)N(N(C1=O)c2ccccc2)c3ccccc3,CCCCC1C(=O)N(N(C1=O)c2ccccc2)c3ccccc3
2,paracetamol,CC(=O)Nc1ccc(O)cc1,CC(=O)Nc1ccc(O)cc1
3,simvastatin,CCC(C)(C)C(=O)O[C@H]1C[C@@H](C)C=C2C=C[C@H](C)[C@H](CC[C@@H]3C[C@@H](O)CC(=O)O3)[C@@H]12,CCC(C)(C)C(=O)O[C@H]1C[C@@H](C)C=C2C=C[C@H](C)[C@H](CC[C@@H]3C[C@@H](O)CC(=O)O3)[C@@H]12
4,acetominophen,None,None
5,methapyrilene,CN(C)CCN(Cc1sccc1)c2ccccn2,CN(C)CCN(Cc1sccc1)c2ccccn2
